## Use Madmom for Chords Recognition

In [ ]:
# Install python libray (TODO: Add to requirement.txt for Docker)

# pip install pandas==1.3.5
# pip install numpy==1.19.5
# pip install scipy==1.10
# pip install matplotlib==3.6
# pip install madmom

# install ffmpeg on Ubuntu
# apt update
# apt install ffmpeg

In [2]:
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import pytorch_lightning as pl
from scipy.spatial.distance import cosine

import sys
sys.path.insert(0, '/app')
from evaluate import load_pitchclass2vec_model

from collections import defaultdict
from itertools import groupby
import re

print("done")
RANDOM_SEED = 42
pl.seed_everything(seed=RANDOM_SEED)

Global seed set to 42


done


42

In [3]:
# 导入所需的模块
import numpy as np
from madmom.features.chords import CNNChordFeatureProcessor, CRFChordRecognitionProcessor
from madmom.processors import SequentialProcessor

# 音频文件路径
audio_file = '/app/jie_test_music/Drake - Passionfruit.mp3'

# 创建一个特征提取器实例
feature_processor = CNNChordFeatureProcessor()

# 创建一个和弦识别器实例
chord_recognizer = CRFChordRecognitionProcessor()

# 将两个处理器串联成一个序列处理器
sequential_processor = SequentialProcessor([feature_processor, chord_recognizer])

# 应用处理器到音频文件上，识别和弦
chords = sequential_processor(audio_file)

# 打印识别出的和弦
for chord in chords:
    start, end, label = chord
    print(f"Start: {start:.2f}, End: {end:.2f}, Chord: {label}")



print("done")

Start: 0.00, End: 4.30, Chord: E:maj
Start: 4.30, End: 7.50, Chord: C#:min
Start: 7.50, End: 10.60, Chord: F#:maj
Start: 10.60, End: 11.70, Chord: D#:min
Start: 11.70, End: 13.00, Chord: B:maj
Start: 13.00, End: 13.70, Chord: G#:min
Start: 13.70, End: 14.10, Chord: D#:min
Start: 14.10, End: 15.20, Chord: F#:maj
Start: 15.20, End: 21.70, Chord: E:maj
Start: 21.70, End: 24.90, Chord: C#:min
Start: 24.90, End: 25.60, Chord: E:maj
Start: 25.60, End: 26.20, Chord: D#:min
Start: 26.20, End: 30.20, Chord: B:maj
Start: 30.20, End: 31.00, Chord: D#:min
Start: 31.00, End: 32.20, Chord: F#:maj
Start: 32.20, End: 38.70, Chord: E:maj
Start: 38.70, End: 41.20, Chord: N
Start: 41.20, End: 42.50, Chord: A:maj
Start: 42.50, End: 47.30, Chord: B:maj
Start: 47.30, End: 48.70, Chord: G#:min
Start: 48.70, End: 49.50, Chord: F#:maj
Start: 49.50, End: 50.70, Chord: N
Start: 50.70, End: 53.60, Chord: E:maj
Start: 53.60, End: 54.70, Chord: C:maj
Start: 54.70, End: 56.10, Chord: E:maj
Start: 56.10, End: 58.20, 

In [4]:
# print the info of chords array
print(len(chords))
print(type(chords))
print(chords[0:3])

117
<class 'numpy.ndarray'>
[(0. ,  4.3, 'E:maj') (4.3,  7.5, 'C#:min') (7.5, 10.6, 'F#:maj')]


In [5]:
from pitchclass2vec.pitchclass2vec import NaiveEmbeddingModel
from pitchclass2vec import encoding, model

encoder = encoding.RootIntervalDataset
embedding_model = NaiveEmbeddingModel(
                        encoding_model=encoder, 
                        embedding_dim=3, # dim=3 because each '24 basic chords' only contain 3 notes
                        norm=False)

 

In [12]:
# from tasks.segmentation.functional import LSTMBaselineModel
from tasks.segmentation.deeplearning_models.lstm import LSTMBaselineModel
import torch 

# Embed the chord by embedding model (store in /out)
def embed_chord(p2v, c):
    try:
        return p2v[c]
    except:
        return p2v["N"]

# Load the pre-trained embedding model
# p2v = load_pitchclass2vec_model("root-interval", "fasttext", "/app/out/root_interval_best/root-interval-fasttext-with-Processed-ChoCo.ckpt")
print("done: loaded the model")

# Load the pre-trained LSTM model for prediction:
CKPT_PATH = '/app/segmentation_out/13_run.ckpt'
model = LSTMBaselineModel.load_from_checkpoint(CKPT_PATH)

# Obtain the embedded chords 
chords_str = [chord[2] for chord in chords]
corpus_embedded = np.stack([
    np.mean(np.array([embed_chord(embedding_model, c) for c in x]), axis=0) for x in chords
])

print(f"corpus_embedded.shape: {corpus_embedded.shape}",end='\n')


# Obtain correct input format: add a batchsize on the first position
corpus_embedded = torch.tensor(corpus_embedded).unsqueeze(0)
print(f"corpus_embedded.shape: {corpus_embedded.shape}",end='\n')
# ------------------------------------------------------------------------------------------------------------

# Evalutaion mode and  Move to GPU
model.eval() 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
corpus_embedded = corpus_embedded.float().to(device) # change float(float 64) into double(float32)



# Prediction: will return (x,y), x is result after classification(x), y is x after softmax(x)
with torch.no_grad():
    predictions = model.evaluation_forward(corpus_embedded)

print(f"Prediction shape: {predictions[1].shape}")



done: loaded the model
corpus_embedded.shape: (117, 3)
corpus_embedded.shape: torch.Size([1, 117, 3])
Result after classification:tensor([[[-1.6412,  0.5245,  0.4142,  ..., -1.6219, -0.5268,  1.4434],
         [-2.1164,  0.9851,  0.2581,  ..., -1.7306, -0.7539,  2.0001],
         [-2.2960,  1.3868,  0.2926,  ..., -1.8313, -0.8097,  2.3418],
         ...,
         [ 1.4268,  2.9531,  2.0472,  ..., -2.0260,  0.0395,  2.1543],
         [ 1.2329,  2.7068,  1.9630,  ..., -1.9274,  0.0187,  1.9478],
         [ 1.0434,  2.2073,  1.7487,  ..., -1.6685,  0.0984,  1.5906]]],
       device='cuda:0')
Result after softmax: tensor([[[-1.6412,  0.5245,  0.4142,  ..., -1.6219, -0.5268,  1.4434],
         [-2.1164,  0.9851,  0.2581,  ..., -1.7306, -0.7539,  2.0001],
         [-2.2960,  1.3868,  0.2926,  ..., -1.8313, -0.8097,  2.3418],
         ...,
         [ 1.4268,  2.9531,  2.0472,  ..., -2.0260,  0.0395,  2.1543],
         [ 1.2329,  2.7068,  1.9630,  ..., -1.9274,  0.0187,  1.9478],
         [ 1.

In [13]:
predicted_labels = torch.argmax(predictions[1], dim=-1)  # shape will be (1, 115)
print(f"Input: chords_str: {chords_str}")
print('\n')
print(f"Output: predicted_labels: {predicted_labels}")

Input: chords_str: ['E:maj', 'C#:min', 'F#:maj', 'D#:min', 'B:maj', 'G#:min', 'D#:min', 'F#:maj', 'E:maj', 'C#:min', 'E:maj', 'D#:min', 'B:maj', 'D#:min', 'F#:maj', 'E:maj', 'N', 'A:maj', 'B:maj', 'G#:min', 'F#:maj', 'N', 'E:maj', 'C:maj', 'E:maj', 'N', 'B:maj', 'E:maj', 'C#:min', 'C#:maj', 'F#:maj', 'B:maj', 'D#:min', 'G#:min', 'F#:maj', 'E:maj', 'C#:min', 'B:maj', 'F#:maj', 'E:maj', 'C#:min', 'F#:maj', 'D#:min', 'B:maj', 'F#:maj', 'E:maj', 'C#:min', 'B:maj', 'D#:min', 'B:maj', 'F#:maj', 'E:maj', 'C#:min', 'F#:maj', 'D#:min', 'B:maj', 'D#:min', 'F#:maj', 'E:maj', 'C#:min', 'F#:maj', 'D#:min', 'B:maj', 'G#:maj', 'E:maj', 'C#:min', 'B:maj', 'D#:min', 'F#:maj', 'E:maj', 'C#:min', 'F#:maj', 'D#:min', 'B:maj', 'D#:min', 'F#:maj', 'E:maj', 'C#:min', 'B:maj', 'D#:min', 'B:maj', 'F#:maj', 'E:maj', 'C#:min', 'F#:maj', 'D#:min', 'B:maj', 'F#:maj', 'E:maj', 'N', 'C#:maj', 'N', 'D#:maj', 'F#:maj', 'D#:maj', 'D#:min', 'B:maj', 'D#:min', 'D#:maj', 'B:maj', 'D#:maj', 'C#:maj', 'F#:maj', 'E:maj', 'C#

In [14]:
label_list = list(predicted_labels)[0]
change_point = []
for i in range(len(label_list)-1):
    if label_list[i] != label_list[i+1]: change_point.append(i)

end_time = [chords[cp][1] for cp in change_point]
print(f"end_time: {end_time}")

end_time: [32.2, 108.30000000000001]


In [ ]:
from sklearn.preprocessing import OneHotEncoder
predicted_indices = torch.argmax(predictions[1], dim=-1)

# 将这些索引转换成独热编码格式
predicted_onehot = torch.zeros_like(predictions[1]).scatter_(-1, predicted_indices.unsqueeze(-1), 1)

# 转换为numpy数组
predicted_onehot = predicted_onehot.cpu().numpy()

# 使用inverse_transform将独热编码转换回原始标签
label_encoder = 
predicted_labels = label_encoder.inverse_transform(predicted_onehot)

print(f"Predicted labels: {predicted_labels}")